In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import os
import _pickle as cPickle
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import numpy as np

torch.set_default_device('cpu') 
device = torch.device('cuda:0')

IMAGENET_PATH = "../data/Imagenet32_train/"
PATH = "../2.NeuralHashing/mainModel.pth"
HASH_SIZE = 64

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

def load_databatch(data_folder, idx, img_size=32):
    data_file = os.path.join(data_folder, 'train_data_batch_')

    dict_ = unpickle(data_file + str(idx))

    images = dict_['data']
    labels = dict_['labels']

    ## images_without_mean = dict_['mean']   ??? check is there are better results or not

    return images, labels

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [3]:
images, labels = load_databatch(IMAGENET_PATH, 1)

In [4]:
print( "size of images:", len(labels) )
images = images.astype(np.float32).reshape(128116,3,32,32)
dataset = CustomDataset(torch.from_numpy(images), torch.Tensor(labels) )
ImageNetLoader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

size of images: 128116


In [5]:
torch.max( torch.Tensor(labels) , 0 )

torch.return_types.max(
values=tensor(1000.),
indices=tensor(392))

In [6]:
class Net(nn.Module):
    def __init__(self, bits):
        super().__init__()
        self.fc1 = nn.Linear(2048, 1000)
        self.hash1 = nn.Linear(1000, 64)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(1000, 1000)

    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.sigmoid(x)
        hash_ = self.hash1(x)
        x = self.fc2(x)
        x = x.softmax(1)
        return x, hash_

net = Net( HASH_SIZE )
net.to(device)

resnet50 = models.resnet50(pretrained = True) # weights=ResNet50_Weights.DEFAULT
resnet50 = resnet50.to(device)
resnet50.fc = torch.nn.Identity()

for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.fc.parameters():
    param.requires_grad = True
    
for param in resnet50.layer4.parameters():
    param.requires_grad = True

model = nn.Sequential(resnet50, net)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
#model.load_state_dict(torch.load(PATH))
model = model.to(device)

criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
iterations = 50

In [8]:
for epoch in range( 10 ):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(ImageNetLoader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer2.zero_grad()
        outputs, hash_ = model(inputs)

        print( torch.max(outputs) )

        #outputs = outputs.type(torch.int64)
        labels_one_hot = torch.nn.functional.one_hot( labels.type(torch.int64), 1000 ).to(torch.float32)  
        loss = criterion2(outputs.to(device), labels_one_hot.to(device))        
        loss.backward()

        optimizer2.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print(i + 1, " ", end="")
            
        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted.to(device) == labels.to(device)).sum().item()

    print(f'[{epoch + 1}] loss: {running_loss} accurancy: {100 * correct // total} %')

/usr/local/lib/python3.10/dist-packages/torch/utils/_device.py:78: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return func(*args, **kwargs)


tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
1  tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0026, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0026, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0026, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0026, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0025, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0024, device='cuda:0', grad_fn=<MaxBack

../aten/src/ATen/native/cuda/ScatterGatherKernel.cu:365: operator(): block: [0,0,0], thread: [10,0,0] Assertion `idx_dim >= 0 && idx_dim < index_size && "index out of bounds"` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model = model.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in ImageNetLoader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs, _ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')